# Smart Hospital Database

Generating a simulated dataset based on the ER Diagram used here : https://www.w3resource.com/sql-exercises/hospital-database-exercise/index.php

In [1]:
# Step 1: Generate hospital blocks
import pandas as pd
import random

# Parameters
NUM_FLOORS = 5
BLOCKS_PER_FLOOR = 4

block_data = []
for floor in range(1, NUM_FLOORS + 1):
    for blockcode in range(1, BLOCKS_PER_FLOOR + 1):
        block_data.append({
            'blockfloor': floor,
            'blockcode': blockcode
        })

df_block = pd.DataFrame(block_data)
df_block.to_csv("block.csv", index=False)
df_block.head()

,blockfloor,blockcode
0,1,1
1,1,2
2,1,3
3,1,4
4,2,1


In [2]:
# Step 2: Generate hospital rooms
import pandas as pd
import random

# Load blocks from previous step
df_block = pd.read_csv("block.csv")

# Parameters
NUM_ROOMS = 100
room_types = ["ICU", "General", "Semi-Private", "Private"]

room_data = []
for room_id in range(1, NUM_ROOMS + 1):
    block = df_block.sample().iloc[0]
    room_data.append({
        'roomnumber': room_id,
        'roomtype': random.choice(room_types),
        'blockfloor': int(block['blockfloor']),
        'blockcode': int(block['blockcode']),
        'unavailable': random.choice([True, False, False])  # More likely available
    })

df_room = pd.DataFrame(room_data)
df_room.to_csv("room.csv", index=False)
df_room.head()

,roomnumber,roomtype,blockfloor,blockcode,unavailable
0,1,General,1,1,False
1,2,Private,1,3,False
2,3,General,5,3,True
3,4,General,4,3,False
4,5,ICU,2,4,False


In [3]:
# Step 3: Generate departments
department_names = [
    "Cardiology", "Neurology", "Orthopedics", "Pediatrics", "Oncology",
    "Emergency", "Radiology", "Dermatology", "Psychiatry", "Gastroenterology"
]

department_data = []
for i, name in enumerate(department_names, start=1):
    department_data.append({
        'departmentid': i,
        'name': name,
        'head': None  # Placeholder, to be updated later with physician IDs
    })

df_department = pd.DataFrame(department_data)
df_department.to_csv("department.csv", index=False)
df_department.head()

,departmentid,name,head
0,1,Cardiology,None
1,2,Neurology,None
2,3,Orthopedics,None
3,4,Pediatrics,None
4,5,Oncology,None


In [4]:
# Step 4: Generate physicians
from faker import Faker
import pandas as pd
import random

fake = Faker()

# Parameters
NUM_PHYSICIANS = 50
positions = [
    "Attending Physician", "Surgeon", "Resident", "Consultant",
    "General Practitioner", "Specialist", "Cardiologist", "Neurologist"
]

physician_data = []
used_ssns = set()

for i in range(1, NUM_PHYSICIANS + 1):
    ssn = random.randint(100000000, 999999999)
    while ssn in used_ssns:
        ssn = random.randint(100000000, 999999999)
    used_ssns.add(ssn)
    
    physician_data.append({
        'employeeid': i,
        'name': fake.name(),
        'position': random.choice(positions),
        'ssn': ssn
    })

df_physician = pd.DataFrame(physician_data)
df_physician.to_csv("physician.csv", index=False)
df_physician.head()

,employeeid,name,position,ssn
0,1,Kelly Crawford,Surgeon,841875042
1,2,Jessica Thomas,Neurologist,405114619
2,3,Thomas Klein,Cardiologist,974151108
3,4,Allen Becker,Attending Physician,289358326
4,5,Katie Campbell,Cardiologist,699825206


In [5]:
# Update department heads with physician IDs
import random

df_department = pd.read_csv("department.csv")
df_physician = pd.read_csv("physician.csv")

# Randomly assign a physician to each department as head
df_department['head'] = df_physician.sample(n=len(df_department))['employeeid'].values
df_department.to_csv("department.csv", index=False)
df_department.head()

,departmentid,name,head
0,1,Cardiology,38
1,2,Neurology,43
2,3,Orthopedics,7
3,4,Pediatrics,34
4,5,Oncology,44


In [6]:
# Step 5: Generate nurses
from faker import Faker
import pandas as pd
import random

fake = Faker()

# Parameters
NUM_NURSES = 60
nurse_roles = [
    "Registered Nurse", "Licensed Practical Nurse",
    "Nurse Practitioner", "Staff Nurse", "Charge Nurse"
]

nurse_data = []
used_ssns = set()

for i in range(1, NUM_NURSES + 1):
    ssn = random.randint(100000000, 999999999)
    while ssn in used_ssns:
        ssn = random.randint(100000000, 999999999)
    used_ssns.add(ssn)

    nurse_data.append({
        'employeeid': i,
        'name': fake.name(),
        'position': random.choice(nurse_roles),
        'registered': random.choice([True, False]),
        'ssn': ssn
    })

df_nurse = pd.DataFrame(nurse_data)
df_nurse.to_csv("nurse.csv", index=False)
df_nurse.head()


,employeeid,name,position,registered,ssn
0,1,Gary Martinez,Nurse Practitioner,False,448012921
1,2,Meghan Ayala,Staff Nurse,True,913756589
2,3,Ronald Smith,Staff Nurse,True,855276321
3,4,Ryan Thompson,Licensed Practical Nurse,False,651440845
4,5,Noah Duke,Registered Nurse,True,710065978


In [7]:
# Step 6: Generate patients
from faker import Faker
import pandas as pd
import random

fake = Faker()

# Load physicians for PCP assignment
df_physician = pd.read_csv("physician.csv")

# Parameters
NUM_PATIENTS = 100
used_ssns = set()

patient_data = []
for _ in range(NUM_PATIENTS):
    ssn = random.randint(100000000, 999999999)
    while ssn in used_ssns:
        ssn = random.randint(100000000, 999999999)
    used_ssns.add(ssn)

    patient_data.append({
        'ssn': ssn,
        'name': fake.name(),
        'address': fake.address().replace("\n", ", "),
        'phone': fake.phone_number(),
        'insuranceid': random.randint(10000, 99999),
        'pcp': df_physician.sample(1)['employeeid'].iloc[0]
    })

df_patient = pd.DataFrame(patient_data)
df_patient.to_csv("patient.csv", index=False)
df_patient.head()


,ssn,name,address,phone,insuranceid,pcp
0,455176690,Jeffrey Moore,"4287 Barnes Inlet Suite 912, New Tiffany, ID 5...",+1-258-340-7552x97112,29245,2
1,840811430,Charles Romero,"69692 Crystal Springs Suite 562, Kevinfort, NY...",001-774-739-8473x7886,51499,47
2,162642654,Billy Hall,"Unit 5027 Box 2201, DPO AP 26012",540-821-2689,71135,43
3,577338063,Nicole Caldwell,"PSC 6860, Box 4521, APO AP 22106",(263)289-9083x924,73315,22
4,905816311,Jon Johnson,"73202 Michelle Groves Suite 600, Kristinfurt, ...",+1-897-892-9546x578,79388,18


In [8]:
# Step 6.5 : Generate procedure
import pandas as pd
import random

# Sample procedure names
procedure_names = [
    "Appendectomy", "CT Scan", "MRI", "Blood Test", "X-Ray",
    "Echocardiogram", "Colonoscopy", "Endoscopy", "Dialysis",
    "Cataract Surgery", "Hip Replacement", "Knee Arthroscopy",
    "Ultrasound", "Biopsy", "Angioplasty", "Chemotherapy",
    "Radiation Therapy", "Pacemaker Implant", "Cesarean Section",
    "Tonsillectomy", "Skin Graft", "Bronchoscopy", "Laparoscopy",
    "Spinal Fusion", "Gastrectomy", "Liver Transplant",
    "Coronary Bypass", "Thyroidectomy", "Mastectomy", "Vasectomy"
]

procedure_data = []

for i, name in enumerate(procedure_names, start=1):
    cost = round(random.uniform(150, 10000), 2)
    procedure_data.append({'code': i, 'name': name, 'cost': cost})

df_procedure = pd.DataFrame(procedure_data)
df_procedure.to_csv("procedure_performed.csv", index=False)
df_procedure.head()


,code,name,cost
0,1,Appendectomy,2293.90
1,2,CT Scan,7866.97
2,3,MRI,1607.10
3,4,Blood Test,1669.98
4,5,X-Ray,6279.61


In [9]:
# Step 7: Generate hospital stays
import pandas as pd
import random
from faker import Faker
from datetime import timedelta

fake = Faker()

# Load patients and available rooms
df_patient = pd.read_csv("patient.csv")
df_room = pd.read_csv("room.csv")
available_rooms = df_room[df_room['unavailable'] == False]['roomnumber'].tolist()

stay_data = []
stay_id = 1

for _, patient in df_patient.iterrows():
    num_stays = random.randint(1, 3)  # Each patient may have 1–3 stays
    for _ in range(num_stays):
        room = random.choice(available_rooms)

        # Generate random stay dates
        start_time = fake.date_time_between(start_date="-1y", end_date="now")
        stay_length_days = random.randint(1, 14)
        end_time = start_time + timedelta(days=stay_length_days)

        stay_data.append({
            'stayid': stay_id,
            'patient': patient['ssn'],
            'room': room,
            'start_time': start_time,
            'end_time': end_time
        })
        stay_id += 1

df_stay = pd.DataFrame(stay_data)
df_stay.to_csv("stay.csv", index=False)
df_stay.head()


,stayid,patient,room,start_time,end_time
0,1,455176690,96,2025-01-16 02:21:50,2025-01-21 02:21:50
1,2,455176690,74,2025-01-13 03:05:01,2025-01-18 03:05:01
2,3,455176690,19,2025-05-15 19:47:27,2025-05-27 19:47:27
3,4,840811430,85,2025-05-17 19:50:42,2025-05-19 19:50:42
4,5,840811430,59,2025-04-05 23:39:01,2025-04-13 23:39:01


In [20]:
# Step 8: Generate appointments
import pandas as pd
import random
from faker import Faker
from datetime import timedelta

fake = Faker()

# Load relevant data
df_patient = pd.read_csv("patient.csv")
df_physician = pd.read_csv("physician.csv")
df_nurse = pd.read_csv("nurse.csv")

appointment_data = []
appointment_id = 1

for _, patient in df_patient.iterrows():
    num_appointments = random.randint(1, 5)
    for _ in range(num_appointments):
        physician_id = df_physician.sample(1)['employeeid'].iloc[0]

        # 50% chance of having a prep nurse
        if random.random() < 0.5:
            prep_nurse_id = int(df_nurse.sample(1)['employeeid'].iloc[0])
        else:
            prep_nurse_id = None

        # Random appointment start time within the last 6 months
        start_time = fake.date_time_between(start_date='-6M', end_date='now')
        duration_minutes = random.randint(30, 90)
        end_time = start_time + timedelta(minutes=duration_minutes)

        appointment_data.append({
            'appointmentid': appointment_id,
            'patient': patient['ssn'],
            'prepnurse': prep_nurse_id,
            'physician': physician_id,
            'start_dt_time': start_time,
            'end_dt_time': end_time,
            'examinationroom': f"ER-{random.randint(1, 50)}"
        })
        appointment_id += 1

df_appointment = pd.DataFrame(appointment_data)
df_appointment['prepnurse'] = df_appointment['prepnurse'].astype('Int64')
df_appointment.to_csv("appointment.csv", index=False)
df_appointment.head()

,appointmentid,patient,prepnurse,physician,start_dt_time,end_dt_time,examinationroom
0,1,455176690,<NA>,31,2025-01-21 21:01:59,2025-01-21 21:45:59,ER-5
1,2,840811430,<NA>,27,2024-12-11 01:14:38,2024-12-11 02:04:38,ER-25
2,3,840811430,36,32,2025-02-14 19:30:38,2025-02-14 20:11:38,ER-26
3,4,840811430,<NA>,30,2025-02-09 14:25:54,2025-02-09 15:29:54,ER-5
4,5,162642654,<NA>,29,2024-12-16 10:31:49,2024-12-16 11:22:49,ER-14


In [11]:
# Step 9: Generate medications and prescriptions
import pandas as pd
import random
from faker import Faker

fake = Faker()

# ---- Generate medications ---- #
NUM_MEDICATIONS = 50
med_names = [fake.unique.word().capitalize() + f"{random.randint(10,99)}" for _ in range(NUM_MEDICATIONS)]
brands = ['Pfizer', 'Novartis', 'Merck', 'Roche', 'Sanofi', 'GSK', 'Lilly']

medications = []
for i in range(1, NUM_MEDICATIONS + 1):
    medications.append({
        'code': i,
        'name': med_names[i-1],
        'brand': random.choice(brands),
        'description': fake.sentence(nb_words=6)
    })

df_medication = pd.DataFrame(medications)
df_medication.to_csv("medication.csv", index=False)

# ---- Generate prescriptions ---- #
df_patient = pd.read_csv("patient.csv")
df_physician = pd.read_csv("physician.csv")
df_appointment = pd.read_csv("appointment.csv")

doses = ["500mg once daily", "250mg twice daily", "100mg at bedtime", "1 tablet after meals", "2 capsules per day"]

prescribes = []
for _, appt in df_appointment.iterrows():
    if random.random() < 0.6:  # 60% chance that the appointment results in a prescription
        prescribes.append({
            'physician': appt['physician'],
            'patient': appt['patient'],
            'medication': random.randint(1, NUM_MEDICATIONS),
            'date': appt['start_dt_time'],
            'appointment': appt['appointmentid'],
            'dose': random.choice(doses)
        })

df_prescribes = pd.DataFrame(prescribes)
df_prescribes.to_csv("prescribes.csv", index=False)

df_medication.head(), df_prescribes.head()

(   code        name     brand  \
 0     1    Expect41    Sanofi   
 1     2  Whatever80     Lilly   
 2     3     Store57    Sanofi   
 3     4     Bring30  Novartis   
 4     5    Budget58    Sanofi   
 
                                          description  
 0               Air data management score him start.  
 1                         Want spring tend consumer.  
 2  Court yourself impact defense result create sh...  
 3                          Want wait bit trade join.  
 4       Record her actually describe kid coach ball.  ,
    physician    patient  medication                 date  appointment  \
 0         31  455176690          43  2025-01-30 15:52:52            1   
 1         30  162642654          23  2025-04-18 02:44:12            9   
 2          7  162642654          17  2025-01-11 13:16:34           10   
 3         16  162642654          40  2025-01-29 10:31:10           12   
 4         23  162642654          45  2025-03-21 21:47:47           13   
 
           

In [1]:
# Step 10: Generate undergoes
import pandas as pd
import random
from faker import Faker
from datetime import timedelta

fake = Faker()

# Load necessary data
df_stay = pd.read_csv("stay.csv")
df_physician = pd.read_csv("physician.csv")
df_nurse = pd.read_csv("nurse.csv")
df_procedure = pd.read_csv("procedure_performed.csv")

undergoes_data = []

for _, stay in df_stay.iterrows():
    num_procedures = random.randint(1, 3)
    for _ in range(num_procedures):
        procedure = df_procedure.sample(1).iloc[0]
        physician = df_physician.sample(1).iloc[0]['employeeid']
        assisting_nurse = df_nurse.sample(1).iloc[0]['employeeid'] if random.random() < 0.7 else None
        
        # Generate procedure date within stay period
        start = pd.to_datetime(stay['start_time'])
        end = pd.to_datetime(stay['end_time'])
        procedure_date = fake.date_time_between(start_date=start, end_date=end)

        undergoes_data.append({
            'patient': stay['patient'],
            'procedure': procedure['code'],
            'stay': stay['stayid'],
            'date': procedure_date,
            'physician': physician,
            'assistingnurse': assisting_nurse
        })

df_undergoes = pd.DataFrame(undergoes_data)
df_undergoes['assistingnurse'] = df_undergoes['assistingnurse'].astype('Int64')
df_undergoes.to_csv("undergoes.csv", index=False)
df_undergoes.head()

,patient,procedure,stay,date,physician,assistingnurse
0,455176690,18,1,2025-01-16 20:13:02,42,53
1,455176690,30,2,2025-01-17 02:38:23,50,<NA>
2,455176690,5,2,2025-01-13 20:39:33,26,44
3,455176690,16,3,2025-05-16 21:52:48,2,<NA>
4,455176690,9,3,2025-05-16 21:32:25,18,<NA>


In [13]:
# Step 11: Generate trained_in
import pandas as pd
import random
from faker import Faker
from datetime import timedelta

fake = Faker()

# Load data
df_physician = pd.read_csv("physician.csv")
df_procedure = pd.read_csv("procedure_performed.csv")

trained_in_data = []
trained_pairs = set()

for _, doc in df_physician.iterrows():
    num_trainings = random.randint(2, 5)
    procedures = df_procedure.sample(num_trainings)['code'].tolist()

    for proc_code in procedures:
        pair_key = (doc['employeeid'], proc_code)
        if pair_key in trained_pairs:
            continue  # Avoid duplicates
        trained_pairs.add(pair_key)

        # Random cert date: 5–15 years ago
        cert_date = fake.date_between(start_date='-15y', end_date='-5y')
        cert_expiry = cert_date + timedelta(days=365 * random.randint(5, 10))

        trained_in_data.append({
            'physician': doc['employeeid'],
            'treatment': proc_code,
            'certificationdate': cert_date,
            'certificationexpires': cert_expiry
        })

df_trained_in = pd.DataFrame(trained_in_data)
df_trained_in.to_csv("trained_in.csv", index=False)
df_trained_in.head()

,physician,treatment,certificationdate,certificationexpires
0,1,23,2014-06-11,2020-06-09
1,1,27,2010-07-17,2019-07-15
2,1,22,2019-03-04,2025-03-02
3,1,9,2011-03-28,2019-03-26
4,1,3,2017-10-05,2025-10-03


In [14]:
# Step 12 : Generate on_call
import pandas as pd
import random
from faker import Faker
from datetime import timedelta

fake = Faker()

# Load nurses and blocks
df_nurse = pd.read_csv("nurse.csv")
df_block = pd.read_csv("block.csv")

on_call_data = []

for _, nurse in df_nurse.iterrows():
    num_shifts = random.randint(5, 10)
    for _ in range(num_shifts):
        block = df_block.sample(1).iloc[0]
        start_time = fake.date_time_between(start_date='-1y', end_date='now')
        shift_length = timedelta(hours=random.randint(8, 12))
        end_time = start_time + shift_length

        on_call_data.append({
            'nurse': nurse['employeeid'],
            'blockfloor': block['blockfloor'],
            'blockcode': block['blockcode'],
            'oncallstart': start_time,
            'oncallend': end_time
        })

df_on_call = pd.DataFrame(on_call_data)
df_on_call.to_csv("on_call.csv", index=False)
df_on_call.head()

,nurse,blockfloor,blockcode,oncallstart,oncallend
0,1,2,2,2024-06-22 15:46:49,2024-06-23 00:46:49
1,1,1,4,2025-01-17 22:14:32,2025-01-18 07:14:32
2,1,2,1,2024-06-11 03:12:12,2024-06-11 13:12:12
3,1,2,4,2024-09-13 02:54:39,2024-09-13 11:54:39
4,1,5,2,2025-04-27 14:54:15,2025-04-27 23:54:15


In [15]:
# Step 13 : Generate affiliated_with
import pandas as pd
import random

# Load physicians and departments
df_physician = pd.read_csv("physician.csv")
df_department = pd.read_csv("department.csv")

affiliations = []
affiliated_pairs = set()

for _, doc in df_physician.iterrows():
    num_affiliations = random.randint(1, 3)
    departments = df_department.sample(num_affiliations)['departmentid'].tolist()
    primary_index = random.randint(0, num_affiliations - 1)

    for idx, dept_id in enumerate(departments):
        key = (doc['employeeid'], dept_id)
        if key in affiliated_pairs:
            continue
        affiliated_pairs.add(key)

        affiliations.append({
            'physician': doc['employeeid'],
            '_department': dept_id,
            'primaryaffiliation': idx == primary_index
        })

df_affiliated = pd.DataFrame(affiliations)
df_affiliated.to_csv("affiliated_with.csv", index=False)
df_affiliated.head()

,physician,_department,primaryaffiliation
0,1,4,True
1,1,3,False
2,1,7,False
3,2,2,True
4,2,7,False
